# Convolutional Neural Network

### Importing the libraries

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
import os


In [15]:
# Set up directories
dataset_path = "dataset"
train_dir = os.path.join(dataset_path, "training_set")
test_dir = os.path.join(dataset_path, "test_set")


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [17]:
train_datagen = ImageDataGenerator(
    rescale=1./255,   # Normalize pixel values
    shear_range=0.2,  # Apply random shearing
    zoom_range=0.2,   # Apply random zoom
    horizontal_flip=True # Allow random flipping
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize test set

training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Resize images to 64x64
    batch_size=32,
    class_mode='binary'    # Binary classification
)


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [19]:
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [21]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [28]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [31]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [33]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [35]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [37]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 

### Step 5 - Output Layer

In [39]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Part 3 - Training the CNN

### Compiling the CNN

In [41]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5439 - loss: 0.6909

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 85ms/step - accuracy: 0.5441 - loss: 0.6908 - val_accuracy: 0.6560 - val_loss: 0.6235
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step - accuracy: 0.6695 - loss: 0.6077 - val_accuracy: 0.7265 - val_loss: 0.5554
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 89ms/step - accuracy: 0.7130 - loss: 0.5581 - val_accuracy: 0.7115 - val_loss: 0.5583
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - accuracy: 0.7425 - loss: 0.5213 - val_accuracy: 0.7540 - val_loss: 0.5125
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 89ms/step - accuracy: 0.7579 - loss: 0.4870 - val_accuracy: 0.7810 - val_loss: 0.4801
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 90ms/step - accuracy: 0.7813 - loss: 0.4562 - val_accuracy: 0.7850 - val_loss: 0.4653
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 93ms/step - accuracy: 0.8075 - loss: 0.4164 - val_accuracy: 0.7875 - val_loss: 0.4567
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 92ms/step - accuracy: 0.8084 - loss: 0.4105 - val_accurac

### Training the CNN on the Training set and evaluating it on the Test set

In [59]:
import tempfile

with tempfile.TemporaryDirectory() as tmpdirname:
    model_path = f"{tmpdirname}/cat_dog_classifier.keras"
    cnn.save(model_path)
    print(f"Model saved to {model_path}")$

Model saved to /var/folders/nf/bsggfnwd2992lm4gvtg853tm0000gn/T/tmpizxff5p5/cat_dog_classifier.keras


## Part 4 - Making a single prediction

In [94]:
def classify_image(image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0) / 255.0
    result = cnn.predict(test_image)
    if result[0][0] > 0.5:
        return 'DOG'
    else:
        return 'CAT'

In [96]:
# Test prediction
single_prediction_folder = "/Users/fuad/Downloads/ML & DL Project/Project - CNN for Image Classification/dataset/single_prediction/"
if os.path.exists(single_prediction_folder):
    for filename in os.listdir(single_prediction_folder):
        image_path = os.path.join(single_prediction_folder, filename)
        prediction = classify_image(image_path)
        print(f"Image {filename} is a {prediction}.")
else:
    print(f"Folder {single_prediction_folder} does not exist.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Image cat_or_dog_1.jpg is a DOG.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Image cat_or_dog_2.jpg is a CAT.
